### This part is hovernet preprocess: After MATLAB part of analysis finished on the MATLAB code, this is for modifying some of the wsi(ndpi)src, roisrc, and ndpisrc so that we're only using the 214 image files that we want to use. The MATLAB all_output_concat was manually made by just copy and pasting contents of the each of the excel output files from the MATLAB workflow. After running hovernet workflow, manually add hovernet result to all_output_concat to match 1090 features.
### These src's are used in the hovernet_json2df.py, just making sure all the src's are correct so that hovernet_json2df.py runs properly!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
import pickle
from tqdm import tqdm

In [ ]:
xlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx'
ndpisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi'
xl = pd.read_excel(xlsrc)
xl_tmp = xl[xl["student score"] > 1]
wsi_list = xl_tmp.filename
# create ndpi path:
wsi_ndpi_list = [os.path.join(ndpisrc, x) for x in wsi_list]
dst_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\desired_wsi'
for filename in wsi_ndpi_list:
    shutil.copy(filename, dst_src)
# create dl path:
dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
wsi_list = [x.replace(".ndpi", ".tif") for x in wsi_list]
wsi_dl_list = [os.path.join(dlsrc, x) for x in wsi_list]
dst_src1 = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\desired_DLmask'
for filename in wsi_dl_list:
    shutil.copy(filename, dst_src1)
# create roi path:
roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723'
wsi_list = [x.replace(".tif", ".png") for x in wsi_list]
wsi_roi_list = [os.path.join(roisrc, x) for x in wsi_list]
dst_src2 = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723\desired_roi'
for filename in wsi_roi_list:
    shutil.copy(filename, dst_src2)

### This part is hovernet postprocess: This code belows processes the generated .pkl file from hovernet_json2df.py to the .xlsx file in a form that we desire (the average and std values over each patient's roi):

In [1]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
pkl_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df'
output_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
pkl_list = [x for x in os.listdir(pkl_src) if x.endswith(".pkl")]
pkl_full_list = [os.path.join(pkl_src,x) for x in pkl_list] # natsorted

In [3]:
# # code to test one excel:
# total_avg_df = pd.DataFrame()
# total_std_df = pd.DataFrame()
#
# with open(pkl_full_list[2], 'rb') as f:
#     print("converting pkl to df:")
#     df = pickle.load(f)
#     df["Ncell"] = 0
#     all_ncell = []
#     for kk in range(df.shape[0]):
#         contour = df["contour"][kk]
#         ncell = len(contour)
#         all_ncell.append(ncell)
#     df["Ncell"] = all_ncell
#
#     ecm_df = df[df.type==10]
#     ecm_df1 = ecm_df.copy()
#     ecm_df2 = ecm_df.copy()
#     ecm_df3 = ecm_df.copy()
#     # aspect ratio > 3, circularity < 0.5, size > 50um2, far from other cells = stromal fibroblast
#     ecm_df1.loc[(ecm_df1.AspectRatio>3) & (ecm_df1.Circularity<0.5) & (ecm_df1.Area>50) & (ecm_df1.dist2nearest>30),'type'] = 12 #spindle-like cell / fibroblasts
#     # #  size < 50um2, far from other cells = stromal immune
#     ecm_df2.loc[(ecm_df2.Area<1000) & (ecm_df2.dist2nearest>30),'type'] = 13 #resident immune cells
#     # # aspect ratio < 1.5?, circulartiy > 0.5, size < 50um2, close to other cells = inflammatory cells
#     ecm_df3.loc[(ecm_df3.AspectRatio<3) & (ecm_df3.Circularity>0.5) & (ecm_df3.Area<100) & (ecm_df3.dist2nearest<30),'type'] = 14
#     ecm_df1 = ecm_df1[ecm_df1.type==12]
#     ecm_df2 = ecm_df2[ecm_df2.type==13]
#     ecm_df3 = ecm_df3[ecm_df3.type==14]
#     df = pd.concat([df,ecm_df1],axis=0)
#     df = pd.concat([df,ecm_df2],axis=0)
#     df = pd.concat([df,ecm_df3],axis=0)
#     dfs_by_type = {}
#     for t in range(2, 15):
#         dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)
#     expanded_dfs = []
#     for tt, df_t in dfs_by_type.items():
#         new_cols = {f"{col}_type{tt}": df_t[col] for col in df_t.columns if col != 'type'}
#         expanded_dfs.append(pd.DataFrame(new_cols))
#
#     all_set = set(np.unique(df["inroi"]))
#     # apply threshold of 12 to aspect ratio and extract distance and drop non float/int columns and calculate avg:
#     for idx in range(len(expanded_dfs)):
#         type_df = expanded_dfs[idx]
#         roi_col = 'inroi_type' + str(idx+2)
#         type_set = set(np.unique(expanded_dfs[idx][roi_col]))
#         diff = all_set - type_set
#         if len(diff) > 0:
#             print("rois {} not included in type {}".format(diff,idx+2))
#
#         ar_col = 'AspectRatio_type' + str(idx+2)
#         type_df[ar_col] = np.where(type_df[ar_col] > 12, np.nan, type_df[ar_col])
#
#
#         # extract distance:
#         dist_col = 'c2t_distance_type' + str(idx+2)
#         distdf = type_df[dist_col]
#         new_df_cols = [dist_col + "_" + str(kk) for kk in range(2,13)]
#         new_df = pd.DataFrame(columns=new_df_cols)
#         new_df = new_df.reindex(range(distdf.shape[0]))
#
#         for idx1 in range(distdf.shape[0]):
#             row = distdf[idx1]
#             if np.any(pd.isnull(row)):
#                 continue
#             for idx2, value in enumerate(row):
#                 new_df.loc[idx1,new_df_cols[idx2]] = value
#
#         type_df = pd.concat([type_df,new_df],axis=1) # concat the extracted distance to original type_df
#
#         # drop non float/int columns:
#         dropped_cols = []
#         for col2 in type_df.columns:
#             if type_df[col2].dtype not in [int, float, "int64","float64"]:
#                 dropped_cols.append(col2)
#                 if col2 == roi_col:
#                     continue
#                 if "c2t_distance_type" in col2 and len(col2.split(str(idx+2))[1]):
#                     continue
#                 type_df.drop(columns=col2,inplace=True)
#         # replace inf's and nan's with nan's and take mean/std and concatenate
#         type_df = type_df.replace([np.inf, -np.inf, np.nan], np.nan)
#         avg_type_df = type_df.groupby(roi_col).mean()
#         std_type_df = type_df.groupby(roi_col).std()
#         total_avg_df = pd.concat([total_avg_df,avg_type_df],axis=1)
#         total_std_df = pd.concat([total_std_df,std_type_df],axis=1)
#         # save


In [4]:
pkl_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df'
output_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
pkl_list = [x for x in os.listdir(pkl_src) if x.endswith(".pkl")]
pkl_full_list = [os.path.join(pkl_src,x) for x in pkl_list]

In [6]:
for i, pkl in tqdm(enumerate(range(len(pkl_list))),desc="Pickle File Processed",colour='red',total = len(pkl_list)):
    total_avg_df = pd.DataFrame()
    total_std_df = pd.DataFrame()
    imID,_ = os.path.splitext(pkl_list[i])
    output_pth = os.path.join(output_src,imID + ".xlsx")
    with open(pkl_full_list[i], 'rb') as f:
        print("converting pkl to df:")
        df = pickle.load(f)
        df["Ncell"] = 0
        all_ncell = []
        for kk in range(df.shape[0]):
            contour = df["contour"][kk]
            ncell = len(contour)
            all_ncell.append(ncell)
        df["Ncell"] = all_ncell

        ecm_df = df[df.type==10]
        ecm_df1 = ecm_df.copy()
        ecm_df2 = ecm_df.copy()
        ecm_df3 = ecm_df.copy()
        # aspect ratio > 3, circularity < 0.5, size > 50um2, far from other cells = stromal fibroblast
        ecm_df1.loc[(ecm_df1.AspectRatio>3) & (ecm_df1.Circularity<0.5) & (ecm_df1.Area>50) & (ecm_df1.dist2nearest>30),'type'] = 12 #spindle-like cell / fibroblasts
        # #  size < 50um2, far from other cells = stromal immune
        ecm_df2.loc[(ecm_df2.Area<1000) & (ecm_df2.dist2nearest>30),'type'] = 13 #resident immune cells
        # # aspect ratio < 1.5?, circulartiy > 0.5, size < 50um2, close to other cells = inflammatory cells
        ecm_df3.loc[(ecm_df3.AspectRatio<3) & (ecm_df3.Circularity>0.5) & (ecm_df3.Area<100) & (ecm_df3.dist2nearest<30),'type'] = 14
        ecm_df1 = ecm_df1[ecm_df1.type==12]
        ecm_df2 = ecm_df2[ecm_df2.type==13]
        ecm_df3 = ecm_df3[ecm_df3.type==14]
        df = pd.concat([df,ecm_df1],axis=0)
        df = pd.concat([df,ecm_df2],axis=0)
        df = pd.concat([df,ecm_df3],axis=0)

        dfs_by_type = {}
        for t in range(2, 15):
            dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)
        expanded_dfs = []
        for tt, df_t in dfs_by_type.items():
            new_cols = {f"{col}_type{tt}": df_t[col] for col in df_t.columns if col != 'type'}
            expanded_dfs.append(pd.DataFrame(new_cols))

        all_set = set(np.unique(df["inroi"]))
        # apply threshold of 12 to aspect ratio and extract distance and drop non float/int columns and calculate avg:
        for idx in range(len(expanded_dfs)):
            type_df = expanded_dfs[idx]
            roi_col = 'inroi_type' + str(idx+2)
            type_set = set(np.unique(expanded_dfs[idx][roi_col]))
            diff = all_set - type_set
            if len(diff) > 0:
                print("rois {} not included out of all rois {} for type {}".format(diff,all_set,idx+2))

            ar_col = 'AspectRatio_type' + str(idx+2)
            type_df[ar_col] = np.where(type_df[ar_col] > 12, np.nan, type_df[ar_col])


            # extract distance:
            dist_col = 'c2t_distance_type' + str(idx+2)
            distdf = type_df[dist_col]
            new_df_cols = [dist_col + "_" + str(kk) for kk in range(2,13)]
            new_df = pd.DataFrame(columns=new_df_cols)
            new_df = new_df.reindex(range(distdf.shape[0]))

            for idx1 in range(distdf.shape[0]):
                row = distdf[idx1]
                if np.any(pd.isnull(row)):
                    continue
                for idx2, value in enumerate(row):
                    new_df.loc[idx1,new_df_cols[idx2]] = value

            type_df = pd.concat([type_df,new_df],axis=1) # concat the extracted distance to original type_df

            # drop non float/int columns:
            dropped_cols = []
            for col2 in type_df.columns:
                if type_df[col2].dtype not in [int, float, "int64","float64"]:
                    dropped_cols.append(col2)
                    if col2 == roi_col:
                        continue
                    if "c2t_distance_type" in col2 and len(col2.split(str(idx+2))[1]):
                        continue
                    type_df.drop(columns=col2,inplace=True)
            # replace inf's and nan's with nan's and take mean/std and concatenate
            type_df = type_df.replace([np.inf, -np.inf, np.nan], np.nan)
            avg_type_df = type_df.groupby(roi_col).mean()
            std_type_df = type_df.groupby(roi_col).std()
            total_avg_df = pd.concat([total_avg_df,avg_type_df],axis=1)
            total_std_df = pd.concat([total_std_df,std_type_df],axis=1)
            # save
        print("saving excel")

    total_avg_df.to_excel(output_pth, sheet_name = 'averages', index=False)
    with pd.ExcelWriter(output_pth, engine='openpyxl', mode='a') as writer:
        # total_avg_df.to_excel(writer, sheet_name='averages', index=False)
        total_std_df.to_excel(writer, sheet_name='std', index=False)

Pickle File Processed:   0%|          | 0/214 [00:00<?, ?it/s]

converting pkl to df:


C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_type_df = type_df.groupby(roi_col).mean()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:85: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  std_type_df = type_df.groupby(roi_col).std()
Pickle File Processed:   0%|          | 1/214 [00:05<18:41,  5.26s/it]

saving excel
converting pkl to df:


C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_type_df = type_df.groupby(roi_col).mean()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:85: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  std_type_df = type_df.groupby(roi_col).std()


...

C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_type_df = type_df.groupby(roi_col).mean()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:85: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  std_type_df = type_df.groupby(roi_col).std()
Pickle File Processed: 100%|█████████▉| 213/214 [26:04<00:04,  4.78s/it]

saving excel
converting pkl to df:


C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_type_df = type_df.groupby(roi_col).mean()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:85: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  std_type_df = type_df.groupby(roi_col).std()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_12976\3288184646.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the f

rois {1} not included out of all rois {1} for type 3
saving excel


Pickle File Processed: 100%|██████████| 214/214 [26:14<00:00,  7.36s/it]


### Now merge all 214 excel sheets of average together to create a one, large dataframe that we can use to concatenate to the overall clue data df. Note that not all of the excel sheets have the same # of columns, since columns with all NAN's get dropped. Just merge by row!

In [7]:
ind_xl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
ind_xl_name = [x for x in os.listdir(ind_xl_src) if x.endswith(".xlsx")]
ind_xl_path = [os.path.join(ind_xl_src,y) for y in ind_xl_name]
total_df = pd.DataFrame()
for _, xlpth in tqdm(enumerate(ind_xl_path),desc="Excel File Processed",colour='red',total = len(ind_xl_path)):
    ind_df = pd.read_excel(xlpth,sheet_name='averages')
    total_df = pd.concat([total_df,ind_df],axis=0)
total_df = total_df.reset_index(drop=True)
save_path = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df'
total_df.to_excel(os.path.join(save_path,"total_avg" + ".xlsx"))

Excel File Processed: 100%|██████████| 214/214 [00:09<00:00, 22.72it/s]


### Then merge all 214 excel sheets of std together to create one, large dataframe we can use to concatenate to th eoverall clue data df.

In [8]:
ind_xl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
ind_xl_name = [x for x in os.listdir(ind_xl_src) if x.endswith(".xlsx")]
ind_xl_path = [os.path.join(ind_xl_src,y) for y in ind_xl_name]
total_df = pd.DataFrame()
for _, xlpth1 in tqdm(enumerate(ind_xl_path),desc="Excel File Processed",colour='red',total = len(ind_xl_path)):
    ind_df = pd.read_excel(xlpth1,sheet_name='std')
    total_df = pd.concat([total_df,ind_df],axis=0)
total_df = total_df.reset_index(drop=True)
save_path = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df'
total_df.to_excel(os.path.join(save_path, "total_std" + ".xlsx"))

Excel File Processed: 100%|██████████| 214/214 [00:05<00:00, 40.58it/s]


### Lastly, calculate CV:

In [10]:
avg_df_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df\total_avg.xlsx'
std_df_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df\total_std.xlsx'
avg_df = pd.read_excel(avg_df_src)
std_df = pd.read_excel(std_df_src)
avg_df = avg_df.drop(columns="Unnamed: 0")
std_df = std_df.drop(columns="Unnamed: 0")
cv_df = std_df.div(avg_df)
cv_df.to_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df\total_cv.xlsx')

### Then manually copy paste these to the overall clue data df!

In [24]:
# some extra code, this is to get labelid for the clue data df:
def repeat_objects(objects, numbers):
    repeated_objects = [obj for obj, num in zip(objects, numbers) for _ in range(int(num))]
    return repeated_objects

labelidsrc = r"\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230329.xlsx"
labeliddf = pd.read_excel(labelidsrc)
labeliddf = labeliddf[labeliddf["student score"] > 1]
labelid = labeliddf["label name"]
repeat = labeliddf["Nroi"]
labelid = list(labelid)
repeat = list(repeat)

repeat_labelid = repeat_objects(labelid,repeat)

In [27]:
repeat_labelid # len = 364

['10-2135',
 '10-163',
 '10-163',
 '10-163',
 '10-163',
 '10-2924',
 '10-2924',
 '10-2924',
 '10-2924',
 '10-3702',
 '10-3702',
 '10-3702',
 '10-3702',
 '10-4775',
 '10-4775',
 '10-4775',
 '10-5202',
 '03-4359',
 '03-4359',
 '05-14314',
 '03-5872',
 '03-5872',
 '02-17359',
 '02-17359',
 '02-17359',
 '02-17359',
 '10-6634',
 '03-18469',
 '03-18775',
 '03-18775',
 '10-6524',
 '10-5202',
 '05-7318',
 '05:9867',
 '05:9840',
 '05:9840',
 '05:9840',
 '05:7839',
 '05:7839',
 '05:7839',
 '05:7839',
 '05:7839',
 '05-6865',
 '05-6865',
 '05:12047',
 '05:12047',
 '05:12047',
 '05:12047',
 '05:12047',
 '05-15878',
 '05-15878',
 '05-15878',
 '05-15878',
 '05:15205',
 '05-368',
 '05-368',
 '05-368',
 '05:2509',
 '05:2509',
 '05:2509',
 '05-3647',
 '04-1652',
 '05:5386',
 '05:5386',
 '05:5386',
 '05:5386',
 '05:7933',
 '05-9193',
 '05:5386',
 '05:5386',
 '05:5386',
 '05:5386',
 '05:12676',
 '04:537',
 '04:537',
 '04-795',
 '04-795',
 '04-795',
 '04-17773',
 '04-8337',
 '05:368',
 '05:368',
 '05:1085'

In [30]:
repeat_labelid = pd.DataFrame(repeat_labelid)
repeat_labelid.to_excel(r"C:\Users\Kevin\Desktop\tmp4.xlsx")

### For now, replace -inf inf nan's in the entire cell with zeros:

In [32]:
dfsrc = r'\\fatherserverdw\Kevin\skin_morphometric_source_code\2D_CLUE_data_220423.xlsx'
df = pd.read_excel(dfsrc)
df = df.replace([np.inf, -np.inf, np.nan], 0)
df.to_excel(dfsrc)

### Code to find Top 5 cohen's d in the top 4 body parts (chest, back, shoulder, leg):

In [2]:
import pandas as pd
import numpy as np

In [123]:
cohensdsrc = r'\\fatherserverdw\Kevin\skin_morphometric_source_code\Feature Age correlation metrics230412_t1.xlsx'
cohensd_df = pd.read_excel(cohensdsrc,sheet_name = "correlation",header=0)
cohensd_df.columns = cohensd_df.iloc[0]
cohensd_df= cohensd_df.drop(0)
cohensd_df = cohensd_df.replace([np.inf, -np.inf, np.nan], 0)
cohensd_df = cohensd_df.iloc[2:]
cohensd_df = cohensd_df[:-1]
corresp_feature = cohensd_df.iloc[:,3]
corresp_feature_type = cohensd_df.iloc[:,1]
cohensd_df.reset_index(drop=True, inplace=True)
cohensd_df
def top_five_argmax(numbers):
    numbers = np.array(numbers)
    numbers = numbers[~np.isnan(numbers)]
    top_five_indices = np.argsort(numbers)[-5:][::-1]
    return top_five_indices

In [124]:
cohensd_df

,NaN,NaN,NaN,NaN,N-all,N-male,N-female,rho-all,rho-male,rho-female,...,cohendlegmiscf,cohendshouldermiscf,cohendbackscalpf,cohendabdomenscalpf,cohendarmscalpf,cohendchestscalpf,cohendfootscalpf,cohendlegscalpf,cohendscalpshoulderf,cohendscalpmiscf
0,3.0,corneum,S1,corneum composition,214,130,84,-0.111345,-0.112377,-0.145393,...,0.892281,0.355492,0.705361,1.321938,0.592415,0.385392,0.504881,0.818205,0.186155,0.529914
1,4.0,spinosum,S1,spinosum composition,214,130,84,0.078967,0.067140,0.108776,...,1.003508,0.413449,0.880740,1.344348,0.875675,0.820038,0.588353,1.026533,0.527439,0.575171
2,5.0,hair shaft,S1,shaft composition,214,130,84,-0.031598,0.015974,-0.099298,...,1.101092,0.637299,0.400995,1.540982,1.123119,0.253474,0.786163,0.396901,0.015954,2.942765
3,6.0,hair follicle,S1,follicle composition,214,130,84,-0.003457,0.018266,-0.021236,...,0.744535,0.406963,0.346069,0.813839,0.983857,0.276840,0.438549,0.074187,0.533472,1.750471
4,7.0,muscle,S1,muscle composition,214,130,84,-0.022528,-0.051448,0.021459,...,1.131033,1.077009,0.317200,0.785400,0.323835,0.519840,0.282867,0.176244,0.066785,1.504887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,1088.0,stromal,S5,nucleus2nerve_CV,214,130,84,0.074158,0.029094,0.143185,...,1.462919,0.779447,0.567444,0.355291,0.216478,0.884643,0.082765,1.121311,0.349574,1.474590
1086,1089.0,stromal,S5,nucleus2blood_CV,214,130,84,-0.000130,-0.030747,0.045480,...,1.526433,1.174105,0.559900,0.150282,0.567785,1.063283,0.398601,1.037710,0.491202,2.664468
1087,1090.0,stromal,S5,nucleus2ecm_CV,214,130,84,0.074461,0.014368,0.161519,...,0.001539,0.310153,1.245933,0.981313,0.587052,0.410757,0.287301,0.619306,0.542393,1.409639
1088,1091.0,stromal,S5,nucleus2fat_CV,214,130,84,0.038031,0.017857,0.088964,...,1.974458,1.684412,0.536668,0.347621,0.819835,1.154440,0.497937,1.100147,0.620183,1.703069


In [125]:
colnames = ["cohendbackchest","cohendbackleg","cohendbackshoulder","cohendchestleg","cohendchestshoulder","cohendlegshoulder"]
concat_df = pd.DataFrame()
concat_type_df = pd.DataFrame()
concat_value_df = pd.DataFrame()
for idx, colname in enumerate(colnames):
    cohensdlist = cohensd_df[colname].iloc[:,0].tolist()
    top_five_idx = top_five_argmax(cohensdlist)
    features = []
    types= []
    cohendvalues = []
    for ids in top_five_idx:
        feature = corresp_feature[ids+3] # 3 headers cut off
        typez = corresp_feature_type[ids+3]
        cohendvalue = cohensdlist[ids]
        features.append(feature)
        types.append(typez)
        cohendvalues.append(cohendvalue)
    feat_df = pd.DataFrame(features,columns=[colname])
    type_df = pd.DataFrame(types,columns=[colname])
    cd_df = pd.DataFrame(cohendvalues,columns=[colname])
    concat_df = pd.concat([concat_df,feat_df],axis=1)
    concat_type_df = pd.concat([concat_type_df,type_df],axis=1)
    concat_value_df = pd.concat([concat_value_df,cd_df],axis=1)
concat_df

,cohendbackchest,cohendbackleg,cohendbackshoulder,cohendchestleg,cohendchestshoulder,cohendlegshoulder
0,absolute total area,dist2nearest,dist2nearest,mean distance from stromal cell to sweat,absolute total area,mean aspect ratio
1,resident_area,dist2nearest,dist2nearest,absolute total area,content per length scale,std distance from fibroblast to sweat
2,dermis area,std distance from fibroblast to sweat,ma,occurrence per length scale,Perimeter_CV,thickness
3,collagen area,std distance from stromal cell to sweat,mean size,MA_CV,std size,Perimeter_CV
4,mean size,mean aspect ratio,local_align_CV,resident_area,Area_CV,MA_CV


In [126]:
concat_type_df

,cohendbackchest,cohendbackleg,cohendbackshoulder,cohendchestleg,cohendchestshoulder,cohendlegshoulder
0,fat,spindle-like,spindle-like,ecm,fat,blood
1,fat,resident immune,resident immune,sweat,fat,ecm
2,dermis,ecm,epidermis,sweat,fat,epidermis
3,ecm,ecm,fat,fat,fat,spindle-like
4,ecm,blood,epidermis,nerve,fat,stromal


In [127]:
concat_value_df

,cohendbackchest,cohendbackleg,cohendbackshoulder,cohendchestleg,cohendchestshoulder,cohendlegshoulder
0,1.199596,1.413352,0.800197,1.194931,1.359500,1.530267
1,1.046386,1.335253,0.786534,1.137965,1.337644,1.376368
2,1.030288,1.331048,0.707429,1.127317,1.259324,1.357472
3,1.004109,1.275885,0.706584,1.114359,1.235020,1.336240
4,1.003277,1.201469,0.656920,1.084366,1.234803,1.290153


In [122]:
cohensdlist

[0.4755150669261206,
 0.3208372461781037,
 0.1985819520759775,
 0.34745799363530905,
 0.08818906919499744,
 0.7179165345031143,
 0.24124347376564903,
 0.09694077063513278,
 0.17293310016752694,
 0.18015865295467773,
 0.37070757582396346,
 0.245269306642587,
 1.3574720788093928,
 0.18706276419375328,
 0.3830410480215653,
 0.27198439474105707,
 0.6269103027896918,
 0.8367876364125645,
 0.8605828973677071,
 0.860000226336717,
 0.8534828779623217,
 0.08037715904532093,
 0.3858081370346802,
 0.36319081978847917,
 0.15292869633743497,
 0.6729539986426071,
 0.8098490185426702,
 0.8112786098576351,
 0.8026050721511582,
 0.7931806711450238,
 0.9874045197722038,
 0.3680827092386827,
 0.01883932529622711,
 0.11497175486671309,
 0.06543180619187702,
 0.11579587988886342,
 0.11926727620385487,
 0.01090979095390867,
 0.07731775972168346,
 0.1594813818213749,
 0.3831368617380217,
 0.06153917050011153,
 0.03949112501462418,
 0.0846497420511844,
 0.10085572875836785,
 0.11585542803902388,
 0.1307447834

In [49]:
cohensd_df

,NaN,NaN,NaN,NaN,N-all,N-male,N-female,rho-all,rho-male,rho-female,...,cohendlegmiscf,cohendshouldermiscf,cohendbackscalpf,cohendabdomenscalpf,cohendarmscalpf,cohendchestscalpf,cohendfootscalpf,cohendlegscalpf,cohendscalpshoulderf,cohendscalpmiscf
0,1.0,0,1,2,214,130,84,0.08681,0.142901,-0.017746,...,1.565801,1.559447,0.094862,0.320903,0.312772,0.742582,1.489539,0.210713,0.064469,1.263541
1,2.0,0,1,2,214,130,84,0.08681,0.142901,-0.017746,...,1.565801,1.559447,0.094862,0.320903,0.312772,0.742582,1.489539,0.210713,0.064469,1.263541
2,3.0,corneum,S1,corneum composition,214,130,84,-0.111345,-0.112377,-0.145393,...,0.892281,0.355492,0.705361,1.321938,0.592415,0.385392,0.504881,0.818205,0.186155,0.529914
3,4.0,spinosum,S1,spinosum composition,214,130,84,0.078967,0.06714,0.108776,...,1.003508,0.413449,0.88074,1.344348,0.875675,0.820038,0.588353,1.026533,0.527439,0.575171
4,5.0,hair shaft,S1,shaft composition,214,130,84,-0.031598,0.015974,-0.099298,...,1.101092,0.637299,0.400995,1.540982,1.123119,0.253474,0.786163,0.396901,0.015954,2.942765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,1089.0,stromal,S5,nucleus2blood_CV,214,130,84,-0.00013,-0.030747,0.04548,...,1.526433,1.174105,0.5599,0.150282,0.567785,1.063283,0.398601,1.03771,0.491202,2.664468
1089,1090.0,stromal,S5,nucleus2ecm_CV,214,130,84,0.074461,0.014368,0.161519,...,0.001539,0.310153,1.245933,0.981313,0.587052,0.410757,0.287301,0.619306,0.542393,1.409639
1090,1091.0,stromal,S5,nucleus2fat_CV,214,130,84,0.038031,0.017857,0.088964,...,1.974458,1.684412,0.536668,0.347621,0.819835,1.15444,0.497937,1.100147,0.620183,1.703069
1091,1092.0,stromal,S5,Ncell_CV,214,130,84,-0.012989,0.107493,-0.156292,...,0.066377,0.895523,0.840878,0.795213,0.376624,0.32059,1.240614,0.116227,1.294035,0.203955


In [131]:
concat_type_df.to_excel(r"C:\Users\Kevin\Desktop\tmp5.xlsx")

In [132]:
dfsrcc = r"\\fatherserverdw\Kevin\skin_morphometric_source_code\Feature Age correlation metrics230412_t1.xlsx"
my_df = pd.read_excel(dfsrcc,sheet_name="age_related_biomarker")
my_df

,bool,type,core,feature
0,False,corneum,S1,corneum composition
1,False,spinosum,S1,spinosum composition
2,False,hair shaft,S1,shaft composition
3,True,hair follicle,S1,follicle composition
4,False,muscle,S1,muscle composition
...,...,...,...,...
1085,False,stromal,S5,nucleus2nerve_CV
1086,False,stromal,S5,nucleus2blood_CV
1087,False,stromal,S5,nucleus2ecm_CV
1088,False,stromal,S5,nucleus2fat_CV


In [133]:
mybool = my_df["bool"]
features = my_df["feature"]
types = my_df["type"]

In [134]:
aging_features = features[mybool]
aging_types = types[mybool]

In [135]:
aging_features

3       follicle composition
5            oil composition
25              anisotropy_A
26              anisotropy_B
27       anisotropy_A_masked
                ...         
1069          orientation_CV
1075                 oriA_CV
1078      nucleus2corneum_CV
1081     nucleus2follicle_CV
1083          nucleus2oil_CV
Name: feature, Length: 124, dtype: object

In [136]:
aging_types

3       hair follicle
5                 oil
25                ecm
26                ecm
27                ecm
            ...      
1069          stromal
1075          stromal
1078          stromal
1081          stromal
1083          stromal
Name: type, Length: 124, dtype: object

In [145]:
hidf = pd.read_pickle(r"\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\composition\OTS_14684_1_he\87142_93286xy7819.pkl")
hidf

,composition
corneum,0.0
spinosum,0.0
hairshaft,0.0
hairfollicle,0.0
smoothmuscle,0.0
oil,0.0
sweat,0.0
nerve,0.0
bloodvessel,0.0
ecm,0.0


In [144]:
hidf

""
